# AML進階分析

## 載入使用套件

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 連結資料庫

In [ ]:
db = pyodbc.connect(DRIVER='{SQL Server Native Client 10.0}',SERVER='(local)',DATABASE='amlrule_A14',uid='sa',pwd='******')

## 匯入Report案件資料

In [ ]:
report = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_A14_Report]',db)

## 匯入篩規則前所有帳戶資料

In [ ]:
A14_ALL = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_RESULT_A14_20180502_0530]',db)

## 進行規則篩選

In [ ]:
# H1_M6 高風險自然人
# L1_M6 低風險自然人
# H2_M6 高風險法人
# L2_M6 低風險法人

H1_M6 = A14_ALL[(A14_ALL.risk_level == 'H') & (A14_ALL.sex_flag == '1') & (A14_ALL.SUM_TX_AMT >= 1500000)]
H1_M6 = H1_M6[(H1_M6.SUM_TX_AMT > 10 * H1_M6.M6_AVG_TX_AMT) | (H1_M6.M6_AVG_TX_AMT == 0)]
L1_M6 = A14_ALL[((A14_ALL.risk_level != 'H') | (A14_ALL.risk_level is None)) & (A14_ALL.sex_flag == '1') & (A14_ALL.SUM_TX_AMT >= 3000000)]
L1_M6 = L1_M6[(L1_M6.SUM_TX_AMT > 10 * L1_M6.M6_AVG_TX_AMT) | (L1_M6.M6_AVG_TX_AMT == 0)]
H2_M6 = A14_ALL[(A14_ALL.risk_level == 'H') & (A14_ALL.sex_flag == '2') & (A14_ALL.SUM_TX_AMT >= 5000000)]
H2_M6 = H2_M6[(H2_M6.SUM_TX_AMT > 10 * H2_M6.M6_AVG_TX_AMT) | (H2_M6.M6_AVG_TX_AMT == 0)]
L2_M6 = A14_ALL[((A14_ALL.risk_level != 'H') | (A14_ALL.risk_level is None)) & (A14_ALL.sex_flag == '2') & (A14_ALL.SUM_TX_AMT >= 10000000)]
L2_M6 = L2_M6[(L2_M6.SUM_TX_AMT > 10 * L2_M6.M6_AVG_TX_AMT) | (L2_M6.M6_AVG_TX_AMT == 0)]

## 決策樹演算法

In [ ]:
#載入決策樹演算法套件
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.externals.six import StringIO
import pydot

In [ ]:
# data['label'] = 0 無告警
# data['label'] = 1 有告警 

# 生成告警標籤 (label = 0)
data['label'] = np.zeros(len(data))
data['label'] = data['label'].astype(int)
# 生成告警標籤 (label = 1)
data.loc[data['answer'].isnull() == 0, 'label'] = 1 

In [ ]:
# 顯示所有告警數量
sum(data.label)

In [ ]:
# 視覺化告警數量
sns.countplot('label',data=data)

In [ ]:
# data['*risk_level'] 整數型態表示的客戶風險類別

# 將客戶 風險類別轉成整數型態表示
data['*risk_level'] = data.risk_level.replace({'L':0,'M':1,'H':2},axis=0)

In [ ]:
# 保留決策樹演算法所要訓練的欄位X及標籤y
X = data.drop(['CASENO','CASE_TYPE','CUST_IDN','MAINTAIN_UNIT','encrypt_generation','M_TYPE','answer','RULE_DTE','risk_level','label'], axis=1)
y = data['label']

In [ ]:
# 分割訓練及測試資料集 (視情況決定是否分割)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [ ]:
# 訓練資料 (尚未進行參數最佳化)
Dtree = DecisionTreeClassifier()
Dtree.fit(X, y)

In [ ]:
# 預測結果
y_scores = pd.DataFrame(Dtree.predict_proba(X))[1]

In [ ]:
# 評估模型效果 AUCROC CURVE
y_true = y
roc_auc_score(y_true, y_scores)

In [ ]:
# 評估模型效果 Confusion Matrix
y_true = y
y_pred = Dtree.predict(X)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

In [ ]:
# 視覺化決策樹演算法規則
dot_data = StringIO()
dot_data = tree.export_graphviz(Dtree, out_file='A14_tree.dot', feature_names = list(X.columns), class_names = ['Alert','Report'])
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('A14_Dtree')

## 重新抽樣後使用決策樹演算法

In [ ]:
# 比較原始標籤數量
count_class_0, count_class_1 = data.label.value_counts()

df_class_0 = data[data['label'] == 0]
df_class_1 = data[data['label'] == 1]

In [ ]:
# Under Sampling重新抽樣方法
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling')
print(df_test_under.label.value_counts())

df_test_under.label.value_counts().plot(kind='bar', title='Count (Label)')

In [ ]:
# 利用上述方法所得資料 再次進行決策樹演算法的訓練 並觀察評估模型表現
under_X = df_test_under.drop(['CASENO','CASE_TYPE','CUST_IDN','MAINTAIN_UNIT','encrypt_generation','answer','RULE_DTE','risk_level','label'], axis = 1)
under_y = df_test_under['label']

Dtree = DecisionTreeClassifier()
Dtree.fit(under_X,under_y)
y_scores = pd.DataFrame(Dtree.predict_proba(under_X))[1]
y_true = under_y
roc_auc_score(y_true, y_scores)
under_y = df_test_under['label']

Dtree = DecisionTreeClassifier()
Dtree.fit(under_X,under_y)
y_scores = pd.DataFrame(Dtree.predict_proba(under_X))[1]
y_true = under_y
roc_auc_score(y_true, y_scores)

In [ ]:
# Over Sampling 重新抽樣方法
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling')
print(df_test_over.label.value_counts())

df_test_over.label.value_counts().plot(kind='bar', title='Count (Label)')

In [ ]:
# 利用上述方法所得資料 再次進行決策樹演算法的訓練 並觀察評估模型表現
over_X = df_test_over.drop(['CASENO','CASE_TYPE','CUST_IDN','MAINTAIN_UNIT','encrypt_generation','M_TYPE','answer','RULE_DTE','risk_level','label'], axis = 1)
over_y = df_test_over['label']

Dtree = DecisionTreeClassifier()
Dtree.fit(over_X,over_y)
y_scores = pd.DataFrame(Dtree.predict_proba(over_X))[1]
y_true = over_y
roc_auc_score(y_true, y_scores)

In [ ]:
# 評估模型效果 Confusion Matrix
y_true = over_y
y_pred = Dtree.predict(over_X)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

In [ ]:
# 評估模型效果 Simple Hold-out Set 
X_train, X_test, y_train, y_test = train_test_split(over_X, over_y, test_size=0.3, random_state=8)

Dtree = DecisionTreeClassifier()
Dtree.fit(X_train, y_train)
y_scores = pd.DataFrame(Dtree.predict_proba(X_test))[1]
y_true = y_test
roc_auc_score(y_true, y_scores)

In [ ]:
# 評估模型效果 K-Fold Cross Validation
kfold_list = []


Dtree = DecisionTreeClassifier()
kf = KFold(n_splits=10, random_state=66, shuffle=True)
i = 1

for train_index, test_index in kf.split(over_X):
    print("Set:", i)
    i = i+1
    
    X_train, X_test = over_X.iloc[train_index], over_X.iloc[test_index]
    y_train, y_test = over_y.iloc[train_index], over_y.iloc[test_index]
    
    Dtree.fit(X_train, y_train)
    y_scores = pd.DataFrame(Dtree.predict_proba(X_test))[1]
    y_true = y_test
    kfold_list.append(roc_auc_score(y_true, y_scores))
    print("ROCAUC_SCORE:", roc_auc_score(y_true, y_scores))
    print("-"*40)
    
print(np.mean(kfold_list), np.std(kfold_list))

In [ ]:
# 視覺化決策樹演算法規則
dot_data = StringIO()
dot_data = tree.export_graphviz(Dtree, out_file='A14_tree_over.dot', feature_names = list(over_X.columns), class_names = ['Alert','Report'])
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('A14_Dtree_over')

## 自編碼離群值偵測

In [ ]:
# 匯入A14未篩選規則前的所有資料
auto = A14_ALL

In [ ]:
data_all = auto
# 將客戶 風險類別轉成整數型態表示
data_all['*risk_level'] = data_all.risk_level.replace({'L':1,'M':2,'H':3},axis=0)

In [ ]:
data_all.columns

In [ ]:
X = data_all.drop(['RULE_DTE','risk_level'],axis=1)
X.set_index('cust_idn',inplace=True)

In [ ]:
X.fillna(int(0),inplace=True)

In [ ]:
# 載入自編碼套件
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 使用自編碼前先進行Featrue Scaling
X_train = MinMaxScaler().fit_transform(X)

In [ ]:
# 決定輸入層維度
input_dim = X_train.shape[1]

In [ ]:
# One Hidden Layer Autoencoder 一層隱藏層自編碼

encoding_dim = 5
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Deep Autoencoder 深度自編碼

input_layer = Input(shape=(input_dim,))
encoded = Dense(input_dim, activation='relu')(input_layer)
encoded = Dense(4, activation='relu')(encoded)
encoded = Dense(2, activation='relu')(encoded)

decoded = Dense(2, activation='relu')(encoded)
decoded = Dense(4, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# 訓練自編碼模型
autoencoder.fit(X_train, X_train, epochs=10, batch_size=100, shuffle=True, validation_data=(X_train, X_train))

In [ ]:
# 計算出每組輸出值和輸入值的min square error
All = MinMaxScaler().fit_transform(X)
pred = autoencoder.predict(All)
mse = np.mean(np.power(All- pred, 2), axis=1)

error = pd.DataFrame({'reconstruction_error':mse})
error['cust_idn'] = X.index
error.set_index('cust_idn',inplace=True)

In [ ]:
error_ALL = pd.concat([error,X], axis = 1)

In [ ]:
error_ALL.head()

In [ ]:
error_ALL.describe()

In [ ]:
# 找出最可疑的前1000個ID
error_list = error.nlargest(1000,'reconstruction_error')
error_list['CUST_IDN'] = error_list.index

In [ ]:
error_list['CUST_IDN'].nunique()

In [ ]:
#用最可疑的前1000個ID去比對Report表 
A14_Ans = pd.merge(error_list, report, how='inner', on='CUST_IDN')

## 產生新的欄位

### 活存

In [ ]:
# 連結資料庫
db = pyodbc.connect(DRIVER='{SQL Server Native Client 10.0}',SERVER='(local)',DATABASE='amlrule',uid='sa',pwd='Test1234')

In [ ]:
# 讀取半年份的活存交易資料
half_year = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_fshtx6M]',db)

In [ ]:
half_year.head() 

In [ ]:
# 整理時間格式
from datetime import datetime, timedelta
half_year['*BUS_DTE'] = half_year['BUS_DTE'].apply(lambda s:datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])))

In [ ]:
# 將時間設定成index
half_year.set_index('*BUS_DTE', inplace = True)

In [ ]:
# tx_amt_sum_month 每月交易金額加總

grouper23 = half_year.groupby('cust_idn').resample('M')['TX_AMT'].sum()

tx_amt_sum_month = grouper23.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_sum_month.columns)+1):
    column_list.append('tx_amt_sum_month' + str(i))

tx_amt_sum_month.columns = column_list

tx_amt_sum_month.to_csv('tx_amt_sum_month.csv')

In [ ]:
# tx_amt_mean_month 每月交易金額平均

grouper24 = half_year.groupby('cust_idn').resample('M')['TX_AMT'].mean()

tx_amt_mean_month = grouper24.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_mean_month.columns)+1):
    column_list.append('tx_amt_mean_month' + str(i))

tx_amt_mean_month.columns = column_list

tx_amt_mean_month.to_csv('tx_amt_mean_month.csv')

In [ ]:
# tr_cnt_week 每週交易次數

In [ ]:
grouper = half_year.groupby('cust_idn').resample('W')['DRCR'].count()

In [ ]:
tr_cnt_week = grouper.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(tr_cnt_week.columns)+1):
    column_list.append('tr_cnt_week' + str(i))

In [ ]:
tr_cnt_week.columns = column_list

In [ ]:
tr_cnt_week.head()

In [ ]:
tr_cnt_week.to_csv('tr_cnt_week.csv')

In [ ]:
# tx_amt_mean_week 每週交易金額平均

In [ ]:
grouper2 = half_year.groupby('cust_idn').resample('W')['TX_AMT'].mean()

In [ ]:
tx_amt_mean_week = grouper2.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(tx_amt_mean_week.columns)+1):
    column_list.append('tx_amt_mean_week' + str(i))

In [ ]:
tx_amt_mean_week.columns = column_list

In [ ]:
tx_amt_mean_week.head()

In [ ]:
tx_amt_mean_week.to_csv('tx_amt_mean_week.csv')

In [ ]:
# tx_amt_std_week 每週交易金額標準差

In [ ]:
grouper3 = half_year.groupby('cust_idn').resample('W')['TX_AMT'].std()

In [ ]:
tx_amt_std_week = grouper3.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(tx_amt_std_week.columns)+1):
    column_list.append('tx_amt_std_week' + str(i))

In [ ]:
tx_amt_std_week.columns = column_list

In [ ]:
tx_amt_std_week.head()

In [ ]:
tx_amt_std_week.to_csv('tx_amt_std_week.csv')

In [ ]:
# tx_amt_sum_week 每週交易金額加總

In [ ]:
grouper4 = half_year.groupby('cust_idn').resample('W')['TX_AMT'].sum()

In [ ]:
tx_amt_sum_week = grouper4.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(tx_amt_sum_week.columns)+1):
    column_list.append('tx_amt_sum_week' + str(i))

In [ ]:
tx_amt_sum_week.columns = column_list

In [ ]:
tx_amt_sum_week.head()

In [ ]:
tx_amt_sum_week.to_csv('tx_amt_sum_week.csv')

In [ ]:
# bal_mean_week 每週帳戶餘額平均

In [ ]:
grouper5 = half_year.groupby('cust_idn').resample('W')['BAL'].mean()

In [ ]:
bal_mean_week = grouper5.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(bal_mean_week.columns)+1):
    column_list.append('bal_mean_week' + str(i))

In [ ]:
bal_mean_week.columns = column_list

In [ ]:
bal_mean_week.head()

In [ ]:
bal_mean_week.to_csv('bal_mean_week.csv')

In [ ]:
# bal_std_week 每週帳戶餘額標準差

In [ ]:
grouper6 = half_year.groupby('cust_idn').resample('W')['BAL'].std()

In [ ]:
bal_std_week = grouper6.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(bal_std_week.columns)+1):
    column_list.append('bal_std_week' + str(i))

In [ ]:
bal_std_week.columns = column_list

In [ ]:
bal_std_week.head()

In [ ]:
bal_std_week.to_csv('bal_std_week.csv')

In [ ]:
# memo_nuni_month 每月交易類別

In [ ]:
grouper7 = half_year.groupby('cust_idn').resample('M')['MEMO'].nunique()

In [ ]:
memo_nuni_month = grouper7.unstack('*BUS_DTE', fill_value = 0)

In [ ]:
column_list = []

for i in range(1, len(memo_nuni_month.columns)+1):
    column_list.append('memo_nuni_month' + str(i))

In [ ]:
memo_nuni_month.columns = column_list

In [ ]:
memo_nuni_month.head()

In [ ]:
memo_nuni_month.to_csv('memo_nuni_month.csv')

## 支存

In [ ]:
half_year_fch = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_fchtx6M]',db)

In [ ]:
half_year_fch['*BUS_DTE'] = half_year_fch['BUS_DTE'].apply(lambda s:datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])))
half_year_fch.set_index('*BUS_DTE', inplace = True)

In [ ]:
half_year_fch.head()

In [ ]:
# tx_amt_mean_month_fch 每月交易金額平均

grouper25 = half_year_fch.groupby('cust_idn').resample('M')['TX_AMT'].mean()

tx_amt_mean_month_fch = grouper25.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_mean_month_fch.columns)+1):
    column_list.append('tx_amt_mean_month_fch' + str(i))

tx_amt_mean_month_fch.columns = column_list

tx_amt_mean_month_fch.to_csv('tx_amt_mean_month_fch.csv')

In [ ]:
# tx_amt_sum_month_fch 每月交易金額加總

grouper27 = half_year_fch.groupby('cust_idn').resample('M')['TX_AMT'].sum()

tx_amt_sum_month_fch = grouper27.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_sum_month_fch.columns)+1):
    column_list.append('tx_amt_mean_2month_fch' + str(i))

tx_amt_sum_month_fch.columns = column_list

tx_amt_sum_month_fch.to_csv('tx_amt_sum_month_fch.csv')

In [ ]:
# tr_cnt_week_fch 每週交易次數

grouper8 = half_year_fch.groupby('cust_idn').resample('W')['DRCR'].count()

tr_cnt_week_fch = grouper8.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tr_cnt_week_fch.columns)+1):
    column_list.append('tr_cnt_week_fch' + str(i))

tr_cnt_week_fch.columns = column_list

tr_cnt_week_fch.head()

In [ ]:
tr_cnt_week_fch.to_csv('tr_cnt_week_fch.csv')

In [ ]:
# tx_amt_mean_week_fch 每週交易金額平均

grouper9 = half_year_fch.groupby('cust_idn').resample('W')['TX_AMT'].mean()

tx_amt_mean_week_fch = grouper9.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_mean_week_fch.columns)+1):
    column_list.append('tx_amt_mean_week_fch' + str(i))

tx_amt_mean_week_fch.columns = column_list

tx_amt_mean_week_fch.head()

In [ ]:
tx_amt_mean_week_fch.to_csv('tx_amt_mean_week_fch.csv')

In [ ]:
# tx_amt_std_week_fch 每周交易金額標準差

grouper10 = half_year_fch.groupby('cust_idn').resample('W')['TX_AMT'].std()

tx_amt_std_week_fch = grouper10.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_std_week_fch.columns)+1):
    column_list.append('tx_amt_std_week_fch' + str(i))

tx_amt_std_week_fch.columns = column_list

tx_amt_std_week_fch.head()

In [ ]:
tx_amt_std_week_fch.to_csv('tx_amt_std_week_fch.csv')

In [ ]:
# tx_amt_sum_week_fch 每週交易金額加總

grouper11 = half_year_fch.groupby('cust_idn').resample('W')['TX_AMT'].sum()

tx_amt_sum_week_fch = grouper11.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_sum_week_fch.columns)+1):
    column_list.append('tx_amt_sum_week_fch' + str(i))

tx_amt_sum_week_fch.columns = column_list

tx_amt_sum_week_fch.head()

In [ ]:
tx_amt_sum_week_fch.to_csv('tx_amt_sum_week_fch.csv')

In [ ]:
# bal_mean_week_fch 每週帳戶餘額平均

grouper12 = half_year_fch.groupby('cust_idn').resample('W')['BAL'].mean()

bal_mean_week_fch = grouper12.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(bal_mean_week_fch.columns)+1):
    column_list.append('bal_mean_week_fch' + str(i))

bal_mean_week_fch.columns = column_list

bal_mean_week_fch.head()

In [ ]:
bal_mean_week_fch.to_csv('bal_mean_week_fch.csv')

In [ ]:
# bal_std_week_fch 每週帳戶餘額標準差

grouper13 = half_year_fch.groupby('cust_idn').resample('W')['BAL'].std()

bal_std_week_fch = grouper13.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(bal_std_week_fch.columns)+1):
    column_list.append('bal_std_week_fch' + str(i))

bal_std_week_fch.columns = column_list

bal_std_week_fch.head()

In [ ]:
bal_std_week_fch.to_csv('bal_std_week_fch.csv')

In [ ]:
# memo_nuni_month_fch 每月交易類別

grouper14 = half_year_fch.groupby('cust_idn').resample('M')['MEMO'].nunique()

memo_nuni_month_fch = grouper14.unstack('*BUS_DTE', fill_value = 0)

column_list = []

for i in range(1, len(memo_nuni_month_fch.columns)+1):
    column_list.append('memo_nuni_month_fch' + str(i))

memo_nuni_month_fch.columns = column_list

memo_nuni_month_fch.head()

In [ ]:
memo_nuni_month_fch.to_csv('memo_nuni_month_fch.csv')

## 外幣

In [ ]:
half_year_vsf = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_vsfcr6M]',db)

In [ ]:
half_year_vsf['*BUS_DATE'] = half_year_vsf['BUS_DATE'].apply(lambda s:datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])))
half_year_vsf.set_index('*BUS_DATE', inplace = True)

In [ ]:
half_year_vsf.head()

In [ ]:
# tx_amt_mean_month_vsf 每月交易金額平均

grouper26 = half_year_vsf.groupby('CUST_IDN').resample('M')['TX_AMT'].mean()

tx_amt_mean_month_vsf = grouper26.unstack('*BUS_DATE', fill_value = 0)

column_list = []
for i in range(1, len(tx_amt_mean_month_vsf.columns)+1):
    column_list.append('tx_amt_mean_month_vsf' + str(i))

tx_amt_mean_month_vsf.columns = column_list

tx_amt_mean_month_vsf.to_csv('tx_amt_mean_month_vsf.csv')

In [ ]:
# tx_amt_sum_month_vsf 每月交易金額加總

grouper28 = half_year_vsf.groupby('CUST_IDN').resample('M')['TX_AMT'].sum()

tx_amt_sum_month_vsf = grouper28.unstack('*BUS_DATE', fill_value = 0)

column_list = []
for i in range(1, len(tx_amt_sum_month_vsf.columns)+1):
    column_list.append('tx_amt_sum_month_vsf' + str(i))

tx_amt_sum_month_vsf.columns = column_list

tx_amt_sum_month_vsf.to_csv('tx_amt_sum_month_vsf.csv')

In [ ]:
# tr_cnt_week_vsf 每週交易次數

grouper15 = half_year_vsf.groupby('CUST_IDN').resample('W')['DRCR'].count()

tr_cnt_week_vsf = grouper15.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(tr_cnt_week_vsf.columns)+1):
    column_list.append('tr_cnt_week_vsf' + str(i))

tr_cnt_week_vsf.columns = column_list

tr_cnt_week_vsf.head()

In [ ]:
tr_cnt_week_vsf.to_csv('tr_cnt_week_vsf.csv')

In [ ]:
# tx_amt_mean_week_vsf 每週交易金額平均

grouper16 = half_year_vsf.groupby('CUST_IDN').resample('W')['TX_AMT'].mean()

tx_amt_mean_week_vsf = grouper16.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_mean_week_vsf.columns)+1):
    column_list.append('tx_amt_mean_week_vsf' + str(i))

tx_amt_mean_week_vsf.columns = column_list

tx_amt_mean_week_vsf.head()

In [ ]:
tx_amt_mean_week_vsf.to_csv('tx_amt_mean_week_vsf.csv')

In [ ]:
# tx_amt_std_week_vsf 每週交易金額標準差

grouper17 = half_year_vsf.groupby('CUST_IDN').resample('W')['TX_AMT'].std()

tx_amt_std_week_vsf = grouper17.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_std_week_vsf.columns)+1):
    column_list.append('tx_amt_std_week_vsf' + str(i))

tx_amt_std_week_vsf.columns = column_list

tx_amt_std_week_vsf.head()

In [ ]:
tx_amt_std_week_vsf.to_csv('tx_amt_std_week_vsf.csv')

In [ ]:
# tx_amt_sum_week_vsf 每週交易金額加總

grouper18 = half_year_vsf.groupby('CUST_IDN').resample('W')['TX_AMT'].sum()

tx_amt_sum_week_vsf = grouper18.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(tx_amt_sum_week_vsf.columns)+1):
    column_list.append('tx_amt_sum_week_vsf' + str(i))

tx_amt_sum_week_vsf.columns = column_list

tx_amt_sum_week_vsf.head()

In [ ]:
tx_amt_sum_week_vsf.to_csv('tx_amt_sum_week_vsf.csv')

In [ ]:
# bal_mean_week_vsf 每週帳戶餘額平均

grouper19 = half_year_vsf.groupby('CUST_IDN').resample('W')['twd_ACT_BAL'].mean()

bal_mean_week_vsf = grouper19.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(bal_mean_week_vsf.columns)+1):
    column_list.append('bal_mean_week_vsf' + str(i))

bal_mean_week_vsf.columns = column_list

bal_mean_week_vsf.head()

In [ ]:
bal_mean_week_vsf.to_csv('bal_mean_week_vsf.csv')

In [ ]:
# bal_std_week_vsf 每週帳戶餘額標準差

grouper20 = half_year_vsf.groupby('CUST_IDN').resample('W')['twd_ACT_BAL'].std()

bal_std_week_vsf = grouper20.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(bal_std_week_vsf.columns)+1):
    column_list.append('bal_std_week_vsf' + str(i))

bal_std_week_vsf.columns = column_list

bal_std_week_vsf.head()

In [ ]:
bal_std_week_vsf.to_csv('bal_std_week_vsf.csv')

In [ ]:
# memo_nuni_month_vsf 每月交易類別

grouper21 = half_year_vsf.groupby('CUST_IDN').resample('M')['MEMO'].nunique()

memo_nuni_month_vsf = grouper21.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(memo_nuni_month_vsf.columns)+1):
    column_list.append('memo_nuni_month_vsf' + str(i))

memo_nuni_month_vsf.columns = column_list

memo_nuni_month_vsf.head()

In [ ]:
memo_nuni_month_vsf.to_csv('memo_nuni_month_vsf.csv')

In [ ]:
# act_ccy_nuni_month_vsf 每月交易幣別

grouper22 = half_year_vsf.groupby('CUST_IDN').resample('M')['ACT_CCY'].nunique()

act_ccy_nuni_month_vsf = grouper22.unstack('*BUS_DATE', fill_value = 0)

column_list = []

for i in range(1, len(act_ccy_nuni_month_vsf.columns)+1):
    column_list.append('act_ccy_nuni_month_vsf' + str(i))

act_ccy_nuni_month_vsf.columns = column_list

act_ccy_nuni_month_vsf.head()

In [ ]:
act_ccy_nuni_month_vsf.to_csv('act_ccy_nuni_month_vsf.csv')

## Xgboost 分類器

In [ ]:
#讀取新產生的所有特徵欄位

act_ccy_nuni_month_vsf = pd.read_csv('act_ccy_nuni_month_vsf.csv')

bal_mean_week = pd.read_csv('bal_mean_week.csv')
bal_mean_week_fch = pd.read_csv('bal_mean_week_fch.csv')
bal_mean_week_vsf = pd.read_csv('bal_mean_week_vsf.csv')

bal_std_week = pd.read_csv('bal_std_week.csv')
bal_std_week_fch = pd.read_csv('bal_std_week_fch.csv')
bal_std_week_vsf = pd.read_csv('bal_std_week_vsf.csv')

memo_nuni_month = pd.read_csv('memo_nuni_month.csv')
memo_nuni_month_fch = pd.read_csv('memo_nuni_month_fch.csv')
memo_nuni_month_vsf = pd.read_csv('memo_nuni_month_vsf.csv')

tr_cnt_week = pd.read_csv('tr_cnt_week.csv')
tr_cnt_week_fch = pd.read_csv('tr_cnt_week_fch.csv')
tr_cnt_week_vsf = pd.read_csv('tr_cnt_week_vsf.csv')

tx_amt_mean_week = pd.read_csv('tx_amt_mean_week.csv')
tx_amt_mean_week_fch = pd.read_csv('tx_amt_mean_week_fch.csv')
tx_amt_mean_week_vsf = pd.read_csv('tx_amt_mean_week_vsf.csv')

tx_amt_std_week = pd.read_csv('tx_amt_std_week.csv')
tx_amt_std_week_fch = pd.read_csv('tx_amt_std_week_fch.csv')
tx_amt_std_week_vsf = pd.read_csv('tx_amt_std_week_vsf.csv')

tx_amt_sum_week = pd.read_csv('tx_amt_sum_week.csv')
tx_amt_sum_week_fch = pd.read_csv('tx_amt_sum_week_fch.csv')
tx_amt_sum_week_vsf = pd.read_csv('tx_amt_sum_week_vsf.csv')

In [ ]:
tx_amt_sum_month = pd.read_csv('tx_amt_sum_month.csv', index_col = 0).fillna(0)
tx_amt_sum_month_fch = pd.read_csv('tx_amt_sum_month_fch.csv', index_col = 0).fillna(0)
tx_amt_sum_month_vsf = pd.read_csv('tx_amt_sum_month_vsf.csv', index_col = 0).fillna(0)

tx_amt_mean_month = pd.read_csv('tx_amt_mean_month.csv', index_col = 0).fillna(0)
tx_amt_mean_month_fch = pd.read_csv('tx_amt_mean_month_fch.csv', index_col = 0).fillna(0)
tx_amt_mean_month_vsf = pd.read_csv('tx_amt_mean_month_vsf.csv', index_col = 0).fillna(0)

In [ ]:
# 設定二元化門檻
thr = np.mean(tx_amt_sum_month.quantile(q = 0.95))
fch_thr = np.mean(tx_amt_sum_month_fch.quantile(q = 0.95))
vsf_thr = np.mean(tx_amt_sum_month_vsf.quantile(q = 0.95))
thr2 = np.mean(tx_amt_mean_month.quantile(q = 0.95))
fch_thr2 = np.mean(tx_amt_mean_month_fch.quantile(q = 0.95))
vsf_thr2 = np.mean(tx_amt_mean_month_vsf.quantile(q = 0.95))

thr, fch_thr, vsf_thr, thr2, fch_thr2, vsf_thr2

In [ ]:
# 生成二元化特徵欄位
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold = thr)
tx_amt_sum_month_ = pd.DataFrame(binarizer.transform(tx_amt_sum_month))

column_list1 = []

for i in range(1, len(tx_amt_sum_month_.columns)+1):
    column_list1.append('tx_amt_sum_month_' + str(i))

tx_amt_sum_month_.columns = column_list1

binarizer2 = Binarizer(threshold = fch_thr)
tx_amt_sum_month_fch_ = pd.DataFrame(binarizer2.transform(tx_amt_sum_month_fch))

column_list2 = []

for i in range(1, len(tx_amt_sum_month_fch_.columns)+1):
    column_list2.append('tx_amt_sum_month_fch_' + str(i))

tx_amt_sum_month_fch_.columns = column_list2

binarizer3 = Binarizer(threshold = vsf_thr)
tx_amt_sum_month_vsf_ = pd.DataFrame(binarizer3.transform(tx_amt_sum_month_vsf))

column_list3 = []

for i in range(1, len(tx_amt_sum_month_vsf_.columns)+1):
    column_list3.append('tx_amt_sum_month_vsf_' + str(i))

tx_amt_sum_month_vsf_.columns = column_list3

binarizer4 = Binarizer(threshold = thr2)
tx_amt_mean_month_ = pd.DataFrame(binarizer4.transform(tx_amt_mean_month))

column_list4 = []

for i in range(1, len(tx_amt_mean_month_.columns)+1):
    column_list4.append('tx_amt_mean_month_' + str(i))

tx_amt_mean_month_.columns = column_list4

binarizer5 = Binarizer(threshold = fch_thr2)
tx_amt_mean_month_fch_ = pd.DataFrame(binarizer5.transform(tx_amt_mean_month_fch))

column_list5 = []

for i in range(1, len(tx_amt_mean_month_fch_.columns)+1):
    column_list5.append('tx_amt_mean_month_fch_' + str(i))

tx_amt_mean_month_fch_.columns = column_list5

binarizer6 = Binarizer(threshold = vsf_thr2)
tx_amt_mean_month_vsf_ = pd.DataFrame(binarizer6.transform(tx_amt_mean_month_vsf))

column_list6 = []

for i in range(1, len(tx_amt_mean_month_vsf_.columns)+1):
    column_list6.append('tx_amt_mean_month_vsf_' + str(i))
    
tx_amt_sum_month_vsf_.columns = column_list6

In [ ]:
# 生成多項式特徵欄位
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True)

_tx_amt_sum_month_ = pd.DataFrame(poly.fit_transform(tx_amt_sum_month_), columns = poly.get_feature_names(tx_amt_sum_month_.columns))

_tx_amt_sum_month_fch_ = pd.DataFrame(poly.fit_transform(tx_amt_sum_month_fch_), columns = poly.get_feature_names(tx_amt_sum_month_fch_.columns))

_tx_amt_sum_month_vsf_ = pd.DataFrame(poly.fit_transform(tx_amt_sum_month_vsf_), columns = poly.get_feature_names(tx_amt_sum_month_vsf_.columns))

_tx_amt_mean_month_ = pd.DataFrame(poly.fit_transform(tx_amt_mean_month_), columns = poly.get_feature_names(tx_amt_mean_month_.columns))

_tx_amt_mean_month_fch_ = pd.DataFrame(poly.fit_transform(tx_amt_mean_month_fch_), columns = poly.get_feature_names(tx_amt_mean_month_fch_.columns))

_tx_amt_mean_month_vsf_ = pd.DataFrame(poly.fit_transform(tx_amt_mean_month_vsf_))#, columns = poly.get_feature_names(tx_amt_mean_month_vsf_.columns))

In [ ]:
_tx_amt_sum_month_.drop(_tx_amt_sum_month_.columns[[0]],axis=1,inplace=True)
_tx_amt_sum_month_fch_.drop(_tx_amt_sum_month_fch_.columns[[0]],axis=1,inplace=True)
_tx_amt_sum_month_vsf_.drop(_tx_amt_sum_month_vsf_.columns[[0]],axis=1,inplace=True)
_tx_amt_mean_month_.drop(_tx_amt_mean_month_.columns[[0]],axis=1,inplace=True)
_tx_amt_mean_month_fch_.drop(_tx_amt_mean_month_fch_.columns[[0]],axis=1,inplace=True)
_tx_amt_mean_month_vsf_.drop(_tx_amt_mean_month_vsf_.columns[[0]],axis=1,inplace=True)

In [ ]:
# 合併所有新特徵欄位
all_features_list = [act_ccy_nuni_month_vsf,bal_mean_week,bal_mean_week_fch,bal_mean_week_vsf,
                     bal_std_week,bal_std_week_fch,bal_std_week_vsf,
                     memo_nuni_month,memo_nuni_month_fch,memo_nuni_month_vsf,
                     tr_cnt_week,tr_cnt_week_fch,tr_cnt_week_vsf,
                     tx_amt_mean_week,tx_amt_mean_week_fch,tx_amt_mean_week_vsf,
                     tx_amt_std_week,tx_amt_std_week_fch,tx_amt_std_week_vsf,
                     tx_amt_sum_week,tx_amt_sum_week_fch,tx_amt_sum_week_vsf,
                    _tx_amt_sum_month_,_tx_amt_sum_month_fch_,_tx_amt_sum_month_vsf_,
                    _tx_amt_mean_month_,_tx_amt_mean_month_fch_,_tx_amt_mean_month_vsf_]

In [ ]:
tx_amt_mean_week['ID'] = tx_amt_mean_week['cust_idn']
all_features = pd.concat(all_features_list, axis=1)

In [ ]:
all_features.fillna(0, inplace = True)

In [ ]:
# 載入A14的上報案件標籤 
Label = pd.read_sql_query('SELECT * FROM [amlrule_A14].[dbo].[DTT_A14_LABEL]',db)

In [ ]:
data = all_features.set_index('ID').drop(['CUST_IDN','cust_idn'],axis=1)

In [ ]:
data['CUST_IDN'] = data.index

In [ ]:
data_ = pd.merge(data, Label, how = 'left', on = 'CUST_IDN')

In [ ]:
# 將上報案件標籤設為1
data_.loc[data_['answer'].isnull() == 0, 'Label'] = 1

In [ ]:
# 初始化XGBoost參數
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=6, learning_rate=0.06, n_estimators=100, gamma=1, 
                      min_child_weight=1, sub_sample=0.7,
                      colsample_bytree=1, colsample_bylevel=1)

In [ ]:
Label.columns

In [ ]:
X = data_.drop(['CASENO', 'CASE_TYPE', 'CUST_IDN', 'MAINTAIN_UNIT',
       'encrypt_generation', 'M_TYPE', 'answer', 'Label'], axis = 1)
y = data_['Label']

In [ ]:
# 訓練XGBoost模型
model.fit(X,y)

In [ ]:
# 評估模型效果 AUCROC CURVE
y_scores = pd.DataFrame(model.predict_proba(X))[1]
y_true = y
roc_auc_score(y_true, y_scores)

In [ ]:
# 評估模型效果 Confusion Matrix
y_true = y
y_pred = model.predict(X)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

## 特徵重要性

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

plot_importance(model, max_num_features = 20)
pyplot.show()